# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [32]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [33]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [34]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [35]:
song_files = get_files('data/song_data')

In [36]:
filepath = song_files[2]

In [37]:
df = pd.read_json(filepath,lines=True)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [38]:
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()[0]
print(song_data)

['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [39]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [40]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()[0]
artist_data

['AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [41]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [42]:
log_files = get_files('data/log_data')

In [43]:
filepath = log_files[0]

In [44]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [45]:
df = df[df['page']=='NextSong']
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [46]:
import datetime
t = df['ts'].apply(lambda x:datetime.datetime.fromtimestamp(x/1000.0))
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [47]:
time_data = (t.apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')),t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday)

column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [48]:
time_df = pd.DataFrame(dict(zip(column_labels,time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07,0,30,48,11,2018,4
1,2018-11-30 01:08:41,1,30,48,11,2018,4
2,2018-11-30 01:12:48,1,30,48,11,2018,4
3,2018-11-30 01:17:05,1,30,48,11,2018,4
4,2018-11-30 01:20:56,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [49]:
time_df.head(1)

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07,0,30,48,11,2018,4


In [50]:
time_table_insert

'INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s,%s,%s,%s,%s,%s,%s)\n'

In [51]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [52]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [53]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [54]:
from datetime import datetime

In [55]:
a = tuple()
a = a + ('123','322')
a

('123', '322')

In [58]:
check_later = []
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        check_later.append([row.song, row.artist, row.length])

    # insert songplay record
    songplay_data = (datetime.fromtimestamp(row['ts']/1000.0).strftime("%Y-%m-%d %H:%M:%S")
                     , row['userId'], row['level'], songid, artistid, row['sessionId']
                     , row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [61]:
check_list = pd.DataFrame(data = check_later,columns=['song','artist','length'])

In [64]:
check_list

,song,artist,length
0,Jim Henson's Dead,Stephen Lynch,182.85669
1,Shell Shock,Manowar,247.56200
2,Women Lose Weight (Feat: Slick Rick),Morcheeba,257.41016
3,Won't Go Home Without You,Maroon 5,231.23546
4,Hey_ Soul Sister,Train,216.76363
5,I'm In Miami Bitch,LMFAO,227.99628
6,Sexy Bitch,DJ Dizzy,221.15220
7,The Cure & The Cause (Dennis Ferrer Remix),Fish Go Deep & Tracey K,377.41669
8,Staring At Me,M83,96.18240
9,The Second You Sleep,Saybia,257.25342


In [63]:
check_list.to_csv('check_list.csv',index=False)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

#### Checking etl.py function docstrings:

In [3]:
from etl import *

In [4]:
help(process_data)

Help on function process_data in module etl:

process_data(cur, conn, filepath, func)
    Iterate through the files in the specified filepath and process the data with specified pre-process functions.
    
    Parameters:
        cur: cursor, 
        conn: connection, 
        filepath: filepath to be specify, 
        func: function to be specify
        
    Returns:
        check_list: data validation result in csv file, which is storing the songs, artists and durations when the records are not in the Sparkify DB

